In [ ]:
import pandas as pd

quick = True

if quick:
    start_year, end_year, chk, sampled, filename = [2005, 2021, False, True, 'df-light.pkl']
else:
    start_year, end_year, chk, sampled, filename = [2005, 2021, True, False, 'df-full.pkl']

from sklearn.model_selection import train_test_split

df = pd.read_pickle(f'./{filename}')
data = df.iloc[:, 1:]
target = df['grav']

# data = data.drop(columns=['an'],axis=1)
data = data[['catv', 'agg', 'dep', 'col', 'catr', 'catu', 'trajet', 'locp', 'circ', 'situ', 'lum', 'age_cls']]

X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=222)
# X_train, y_train, X_test, y_test = pt.train_test_split_along_time(data, target, 2018)

In [ ]:
import time
from my_libs.encoder_custom import EncoderCustom

start_time = time.time()

cols_target_encoded = ['dep']
cols_onehot_encoded = X_train.columns.drop(cols_target_encoded)

encoder = EncoderCustom(cols_target_encoded=cols_target_encoded, cols_onehot_encoded=cols_onehot_encoded)

X_train_enc, y_train_enc = encoder.transform(X_train, y_train, 'Train')
X_test_enc,  y_test_enc  = encoder.transform(X_test,  y_test,  'Test')

print("--- Features encoding performed in %s seconds ---" % (time.time() - start_time))

In [ ]:
from my_libs.model_evaluator import ModelEvaluator

model_type = 'DecisionTreeClassifier'
# model_type = 'RandomForestClassifier'
# model_type = 'GradientBoostingClassifier'

if model_type == 'DecisionTreeClassifier':
    best_params = {'max_depth': 11, 'criterion': 'gini', 'max_features': 'auto', 'min_samples_split': 4}

if model_type == 'RandomForestClassifier':
    best_params = {'criterion': 'gini', 'max_depth': 20, 'min_samples_split': 2, 'n_estimators': 50}

if model_type == 'GradientBoostingClassifier':
    best_params = {'learning_rate': 0.22, 'n_estimators': 51}

evaluator = ModelEvaluator(model_type=model_type, params=best_params, X_train=X_train_enc, y_train=y_train_enc, X_test=X_test_enc, y_test=y_test_enc)
model = evaluator.evaluate()

In [ ]:
from shapash import SmartExplainer
import shap
shap.initjs()

xpl = SmartExplainer(
    model=model,
    # features_dict=house_dict,  # Optional parameter
    # preprocessing=encoder, # Optional: compile step can use inverse_transform method
    # postprocessing=postprocess # Optional: see tutorial postprocessing
)

y_test_enc.index = X_test_enc.index

xpl.compile(
    x=X_test_enc,
    # y_pred=y_pred, # Optional: for your own prediction (by default: model.predict)
    y_target=y_test_enc, # Optional: allows to display True Values vs Predicted Values
)

app = xpl.run_app(title_story=f"{model_type}", port=8020)

In [ ]:
# app.kill()